In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Conv Block A, added L2 regularization
def conv_block_a(filters, input_shape=None):
    layers = [
        SeparableConv2D(filters, kernel_size=3, strides=3, activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3)  # Add dropout here as well
    ]
    block = Sequential(layers)
    return block

# Conv Block B, added dropout
def conv_block_b(filters):
    block = Sequential([
        SeparableConv2D(filters, kernel_size=3, strides=3, activation='relu', padding='same'),
        BatchNormalization(),
        Dropout(0.3)  
    ])
    return block

# Conv Block C
def conv_block_c():
    block = Sequential([
        SeparableConv2D(512, kernel_size=3, strides=3, activation='relu', padding='same'),
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='sigmoid')
    ])
    return block

# Load your normalized data
X = np.load('normalized_packet_data/452x452_normalized.npy')  

X = np.expand_dims(X, axis=-1)

# Load labels and convert them to integers
labels_df = pd.read_csv('labels/452x452.csv')  
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_df['Label'].values)
y = to_categorical(integer_encoded)

# Define input shape based on X
input_shape = X.shape[1:]

# Building the CNN
model = Sequential()

# Adding Conv Block A with input shape
model.add(conv_block_a(256, input_shape=input_shape))

# Adding 5 Conv Block B
for _ in range(5):
    model.add(conv_block_b(256))

# Adding Conv Block C
model.add(conv_block_c())

# Compiling the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with Early Stopping
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/10
16/16 [==============================] - 19s 952ms/step - loss: 0.5870 - accuracy: 0.8622 - val_loss: 0.4712 - val_accuracy: 0.8516
Epoch 2/10
16/16 [==============================] - 15s 917ms/step - loss: 0.3765 - accuracy: 0.8839 - val_loss: 0.4240 - val_accuracy: 0.8516
Epoch 3/10
16/16 [==============================] - 15s 914ms/step - loss: 0.2836 - accuracy: 0.8839 - val_loss: 0.4216 - val_accuracy: 0.8516
Epoch 4/10
16/16 [==============================] - 15s 923ms/step - loss: 0.1559 - accuracy: 0.8839 - val_loss: 0.7260 - val_accuracy: 0.8516
Epoch 5/10
16/16 [==============================] - 15s 920ms/step - loss: 0.0998 - accuracy: 0.9114 - val_loss: 1.0838 - val_accuracy: 0.8516
Epoch 6/10
16/16 [==============================] - 15s 921ms/step - loss: 0.0766 - accuracy: 0.9921 - val_loss: 1.3279 - val_accuracy: 0.8516
Epoch 7/10
16/16 [==============================] - 15s 920ms/step - loss: 0.0330 - accuracy: 0.9980 - val_loss: 1.4648 - val_accuracy: 0.8516

In [5]:
# Model Summary
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_19 (Sequential)  (None, 75, 75, 256)       1545      
                                                                 
 sequential_20 (Sequential)  (None, 25, 25, 256)       69120     
                                                                 
 sequential_21 (Sequential)  (None, 9, 9, 256)         69120     
                                                                 
 sequential_22 (Sequential)  (None, 3, 3, 256)         69120     
                                                                 
 sequential_23 (Sequential)  (None, 1, 1, 256)         69120     
                                                                 
 sequential_24 (Sequential)  (None, 1, 1, 256)         69120     
                                                                 
 sequential_25 (Sequential)  (None, 2)               

In [6]:
import os

# Define the directory to save the model
model_dir = 'models'

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model
model_save_path = 'models/452x452_tow-ids_model_no_wavelet.h5'

# Save the model in HDF5 format
model.save(model_save_path, save_format='h5')

print(f"Model saved to {model_save_path}")

Model saved to models/452x452_tow-ids_model_no_wavelet.h5


/home/lnagana/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
test_data = np.load('normalized_packet_data/452x452_normalized.npy')
labels_df = pd.read_csv('labels/452x452.csv')
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_df['Label'].values)
test_labels = to_categorical(integer_encoded)

sample_data = test_data[240] 
sample_data = np.expand_dims(X, axis=-1)
sample_label = test_labels[240]

# Make prediction
sample_prediction = model.predict(sample_data)

# Convert predictions to class labels
predicted_class = np.argmax(sample_prediction, axis=1)
true_class = np.argmax(sample_label)  # No need to use axis=1 here as it's a single label

# Assuming label_encoder is a fitted LabelEncoder instance
predicted_class_label = label_encoder.inverse_transform(predicted_class)[0]
true_class_label = label_encoder.inverse_transform([true_class])[0]

print(f"Predicted Class: {predicted_class_label}")
print(f"True Class: {true_class_label}\n")


20/20 [==============================] - 4s 190ms/step
Predicted Class: Normal
True Class: Normal

